# load speech and language embedding

In [1]:
import torch

model_path = '/work/van-speech-nlp/jindaznb/jslpnb/mllm_experiments/slam-llm/examples/asr_librispeech/plots/embeddings/ami_phoneme_wavlm_llama32_1b_linear_peft.pt'
model = torch.load(model_path)

In [2]:
# Assuming 'model' is your dictionary
for key in model.keys():
    print(f"Key: {key}, Shape: {model[key].shape}")

Key: speech_embeddings, Shape: torch.Size([2, 36, 5120])
Key: language_embeddings, Shape: torch.Size([2, 36, 2048])


# load find ckpt path

In [20]:
import os
import torch

# Directory path where the folders are located
dir_path = '/work/van-speech-nlp/jindaznb/jslpnb/mllm_experiments/slam-llm/out/ami_wavlm_llama32_1b_linear_peft'

# Get all directories starting with 'asr_epoch_'
folders = [f for f in os.listdir(dir_path) if f.startswith('asr_epoch_')]

# Sort folders by the numeric value of the loss (7th part of the folder name)
sorted_folders = sorted(folders, key=lambda x: float(x.split('_')[6]))

# The folder with the lowest loss value
lowest_loss_folder = sorted_folders[0]

# Construct the full path
full_path = os.path.join(dir_path, lowest_loss_folder)

# Define the model file path
checkpoint_path = os.path.join(full_path, 'model.pt')
checkpoint_path


'/work/van-speech-nlp/jindaznb/jslpnb/mllm_experiments/slam-llm/out/ami_wavlm_llama32_1b_linear_peft/asr_epoch_2_step_20227_loss_0.6369916200637817/model.pt'

# load ckpt

In [21]:
import os
import torch

# Check if the model file exists and inspect its contents
if os.path.exists(checkpoint):
    checkpoint = torch.load(checkpoint_path, map_location='cpu')  # Load the model file
    print(f"Model loaded successfully from {checkpoint_path}")
    
    # Check if the loaded object is a dictionary (common for PyTorch checkpoints)
    if isinstance(model, dict):
        print("Keys in the model checkpoint:")
        for key in model.keys():
            print(f"- {key}")
    else:
        print("The loaded model is not a dictionary. Printing the object type:")
        print(type(model))
else:
    print("Model file not found.")

Model loaded successfully from /work/van-speech-nlp/jindaznb/jslpnb/mllm_experiments/slam-llm/out/ami_wavlm_llama32_1b_linear_peft/asr_epoch_2_step_20227_loss_0.6369916200637817/model.pt
The loaded model is not a dictionary. Printing the object type:
<class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>


In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(
    '/work/van-speech-nlp/jindaznb/jslpnb/mllm_experiments/slam-llm/models/Llama-3.2-1B-Instruct',
    trust_remote_code=True
)

In [27]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm):

In [14]:
tokenizer = AutoTokenizer.from_pretrained(
            '/work/van-speech-nlp/jindaznb/jslpnb/mllm_experiments/slam-llm/models/Llama-3.2-1B-Instruct', trust_remote_code=True)

In [26]:
llm_checkpoint = {k: v for k, v in checkpoint.items() if 'llm.' in k}
llm_checkpoint

{'llm.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight': tensor([[ 2.5326e-02,  2.8484e-02,  1.3398e-03,  ...,  3.1131e-03,
          -1.0659e-02,  1.6014e-03],
         [ 2.0980e-03,  6.1089e-05,  2.3090e-02,  ...,  4.0515e-02,
          -2.5970e-03, -6.9620e-03],
         [ 1.5990e-02,  3.0519e-02,  1.9506e-02,  ...,  4.0012e-02,
          -1.6976e-02, -3.2935e-02],
         ...,
         [ 1.7744e-02,  2.5626e-02,  2.6607e-02,  ...,  6.8131e-03,
          -6.7438e-03,  2.4509e-03],
         [-2.5725e-02, -1.8041e-02,  9.6223e-03,  ..., -2.9973e-02,
           5.1318e-03, -9.5922e-03],
         [ 3.2651e-02,  2.1047e-02,  2.2146e-02,  ...,  1.8839e-02,
          -3.1481e-02, -4.7576e-03]]),
 'llm.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight': tensor([[-0.0169, -0.0162,  0.0109,  ..., -0.0257,  0.0238, -0.0166],
         [ 0.0159,  0.0135, -0.0033,  ...,  0.0059, -0.0145,  0.0089],
         [ 0.0096,  0.0111,  0.0029,  ...,  0.0141, -0.0

In [15]:
eos_token_id=128009
tokenizer.pad_token_id = tokenizer.eos_token_id

In [25]:
model.load_state_dict(llm_checkpoint, strict=False)

_IncompatibleKeys(missing_keys=['model.embed_tokens.weight', 'model.layers.0.self_attn.q_proj.weight', 'model.layers.0.self_attn.k_proj.weight', 'model.layers.0.self_attn.v_proj.weight', 'model.layers.0.self_attn.o_proj.weight', 'model.layers.0.mlp.gate_proj.weight', 'model.layers.0.mlp.up_proj.weight', 'model.layers.0.mlp.down_proj.weight', 'model.layers.0.input_layernorm.weight', 'model.layers.0.post_attention_layernorm.weight', 'model.layers.1.self_attn.q_proj.weight', 'model.layers.1.self_attn.k_proj.weight', 'model.layers.1.self_attn.v_proj.weight', 'model.layers.1.self_attn.o_proj.weight', 'model.layers.1.mlp.gate_proj.weight', 'model.layers.1.mlp.up_proj.weight', 'model.layers.1.mlp.down_proj.weight', 'model.layers.1.input_layernorm.weight', 'model.layers.1.post_attention_layernorm.weight', 'model.layers.2.self_attn.q_proj.weight', 'model.layers.2.self_attn.k_proj.weight', 'model.layers.2.self_attn.v_proj.weight', 'model.layers.2.self_attn.o_proj.weight', 'model.layers.2.mlp.gat

In [4]:
import torch
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer

# Load a pretrained LLM and tokenizer
model_name = "full_path"  # Replace with your specific LLM model
llm = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Assume we have a learned speech embedding (output of the projector)
# In practice, this would come from your speech embedding model
learned_speech_embedding = torch.randn(1, llm.config.hidden_size)
print(learned_speech_embedding)

# Get the LLM's token embeddings
llm_embeddings = llm.get_input_embeddings().weight

# Compute cosine similarity between the learned embedding and all token embeddings
similarities = F.cosine_similarity(learned_speech_embedding, llm_embeddings, dim=1)

# Find the index of the most similar token
closest_token_idx = similarities.argmax().item()

# Get the actual token (not decoded text) from the tokenizer's vocabulary
closest_token = tokenizer.convert_ids_to_tokens([closest_token_idx])[0]

print(f"Most similar token: {closest_token}")
print(f"Token ID: {closest_token_idx}")

OSError: full_path is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer

# Load a custom LLaMA 3.2 1B model and tokenizer from a checkpoint
model_name = "path_to_llama_model"  # Replace with the local path or model hub path for LLaMA 3.2 1B
llm = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Assume we have a learned speech embedding (output of the projector)
# In practice, this would come from your speech embedding model
learned_speech_embedding = torch.randn(1, llm.config.hidden_size)
print(learned_speech_embedding)

# Get the LLaMA model's token embeddings
llm_embeddings = llm.get_input_embeddings().weight

# Compute cosine similarity between the learned embedding and all token embeddings
similarities = F.cosine_similarity(learned_speech_embedding, llm_embeddings, dim=1)

# Find the index of the most similar token
closest_token_idx = similarities.argmax().item()

# Get the actual token (not decoded text) from the tokenizer's vocabulary
closest_token = tokenizer.convert_ids_to_tokens([closest_token_idx])[0]

print(f"Most similar token: {closest_token}")
print(f"Token ID: {closest_token_idx}")